## 1. Introducción

Este documento presenta un sistema de preparación de datos para implementar Retrieval-Augmented Generation (RAG). El sistema transforma datos en formato CSV a múltiples formatos optimizados para diferentes estrategias de embeddings y búsqueda semántica.

### 1.1 Concepto de RAG

RAG es una técnica que combina dos componentes:

**Retrieval (Recuperación):**
- Búsqueda de documentos relevantes en una base de conocimiento
- Utiliza similitud semántica mediante embeddings
- Encuentra contexto específico del dominio

**Generation (Generación):**
- Modelo de lenguaje genera respuestas
- Utiliza documentos recuperados como contexto
- Reduce alucinaciones y mejora precisión

### 1.2 Ventajas del RAG

**Sobre Fine-Tuning:**
- No requiere reentrenamiento del modelo
- Actualización instantánea de conocimiento
- Menor costo computacional
- Trazabilidad de fuentes

**Sobre Prompts Simples:**
- Acceso a conocimiento específico no presente en el modelo
- Respuestas fundamentadas en documentos reales
- Manejo de dominios especializados
- Escalabilidad con grandes corpus

## 2. Dataset Base

### 2.1 Estructura del Dataset

El sistema trabaja con un dataset CSV que contiene publicaciones de redes sociales:

| Campo | Tipo | Descripción |
|-------|------|-------------|
| `id` | int | Identificador único del documento |
| `fecha` | date | Fecha de publicación |
| `usuario` | string | ID del usuario autor |
| `texto` | string | Contenido textual del documento |
| `tema` | string | Categoría temática |
| `sentimiento` | string | Análisis de sentimiento (positivo/neutral/negativo) |
| `likes` | int | Número de interacciones positivas |
| `reposts` | int | Número de compartidos |

### 2.2 Temáticas del Corpus

El dataset abarca tres temas principales:

**Generación Z y Crisis de Sentido:**
- Búsqueda de propósito en la era digital
- Presión por productividad constante
- Vacío existencial en hiperconectividad
- Expectativas vs. realidad

**Cultura de lo Efímero y Proyectos de Vida:**
- Inmediatez vs. compromisos duraderos
- Identidad fluida y cambios acelerados
- Recompensas instantáneas vs. paciencia
- Construcción de proyectos a largo plazo

**IA y Pérdida de Autonomía Humana:**
- Algoritmos y libertad de elección
- Delegación de decisiones a la tecnología
- Predicción de comportamiento
- Balance entre asistencia y dependencia

### 2.3 Estadísticas del Dataset

**Volumen:**
- Dataset original: 381 documentos
- Dataset ampliado: 5,000 documentos

**Engagement:**
- Promedio de likes por documento
- Promedio de reposts por documento
- Métrica combinada de engagement

**Distribución:**
- Balance entre temas
- Distribución de sentimientos
- Longitud promedio de textos

## 3. Formatos de Salida

El sistema genera cuatro formatos diferentes, cada uno optimizado para un caso de uso específico:

### 3.1 Corpus Simple (corpus_simple.json)

**Estructura:**
```json
{
  "id": 1,
  "texto": "Contenido limpio sin enriquecimiento",
  "metadata": {
    "fecha": "2022-05-01",
    "tema": "Categoría",
    "sentimiento": "neutral",
    "likes": 1056,
    "repostos": 2652
  }
}
```

**Características:**
- Texto original sin modificaciones
- Metadata separada del contenido
- Óptimo para embeddings puros

**Casos de Uso:**
- Modelos de embeddings que separan texto y metadata
- Búsqueda basada únicamente en contenido semántico
- Análisis de similitud sin sesgos de categorización

### 3.2 Corpus Enriquecido (corpus_enriquecido.json)

**Estructura:**
```json
{
  "id": 1,
  "texto": "[Tema] [Sentimiento] Contenido original",
  "texto_original": "Contenido sin modificar",
  "metadata": {...}
}
```

**Características:**
- Prefijos con tema y sentimiento
- Preserva texto original
- Enriquecimiento contextual en el embedding

**Casos de Uso:**
- Búsqueda que considera categorías
- Filtrado implícito por tema o sentimiento
- Mejora de precisión con contexto adicional

### 3.3 Textos Planos (textos_planos.txt)

**Estructura:**
```
Texto del documento 1
Texto del documento 2
Texto del documento 3
...
```

**Características:**
- Un documento por línea
- Sin metadatos ni estructura JSON
- Formato minimalista

**Casos de Uso:**
- Modelos que requieren entrada en texto plano
- Procesamiento batch simple
- Herramientas de línea de comandos

### 3.4 Corpus RAG (corpus_rag.json)

**Estructura:**
```json
{
  "id": "doc_1",
  "content": "Texto limpio",
  "enriched_content": "Tema: X\nSentimiento: Y\n\nTexto",
  "metadata": {
    "engagement": 3708,
    ...
  }
}
```

**Características:**
- Dos versiones del contenido
- ID con prefijo descriptivo
- Métrica de engagement combinada
- Formato estructurado para RAG

**Casos de Uso:**
- Sistemas RAG completos
- Permite elegir entre contenido simple o enriquecido
- Ordenamiento por relevancia (engagement)
- Visualización de fuentes al usuario

## 4. Pipeline de Transformación

### 4.1 Flujo General

```
┌─────────────────────────────────────────┐
│     Dataset CSV Original                │
│  (5000 documentos con metadata)         │
└─────────────┬───────────────────────────┘
              │
              ├─────────────────────────────┐
              │                             │
              ▼                             ▼
    ┌─────────────────┐         ┌──────────────────┐
    │ Texto Simple    │         │ Texto Enriquecido│
    │ (sin modificar) │         │ (con prefijos)   │
    └────────┬────────┘         └────────┬─────────┘
             │                           │
    ┌────────┴────────┐         ┌────────┴─────────┐
    ▼                 ▼         ▼                  ▼
corpus_simple   textos_planos  corpus_enriquecido  corpus_rag
```

### 4.2 Función: optimizar_para_embeddings()

**Propósito:**
- Convertir CSV a formato JSON estructurado
- Opción de enriquecimiento textual
- Preservar metadata relevante

**Parámetros:**
- `csv_path`: Ruta del archivo CSV de entrada
- `output_json`: Ruta del archivo JSON de salida
- `incluir_metadata`: Boolean para enriquecimiento

**Proceso:**
1. Carga CSV con pandas
2. Itera sobre cada fila
3. Extrae y limpia texto
4. Si incluir_metadata=True: añade prefijos [Tema] [Sentimiento]
5. Estructura en formato JSON
6. Guarda resultado

### 4.3 Función: extraer_solo_textos()

**Propósito:**
- Extraer únicamente el contenido textual
- Formato minimalista una línea por documento

**Características:**
- Sin estructura JSON
- Sin metadatos
- Limpieza de espacios
- Salida en texto plano

### 4.4 Función: crear_corpus_busqueda()

**Propósito:**
- Generar formato especializado para RAG
- Combinar ventajas de ambos enfoques

**Enriquecimientos:**
- Dos versiones del contenido (simple y enriquecido)
- Engagement combinado (likes + reposts)
- ID descriptivo con prefijo
- Formato legible por humanos en enriched_content

### 4.5 Función: stats_dataset()

**Propósito:**
- Análisis exploratorio del dataset
- Validación de calidad de datos

**Métricas Reportadas:**
- Total de documentos
- Distribución de temas
- Distribución de sentimientos
- Promedios de engagement
- Longitud promedio de textos

## 5. Embeddings y Búsqueda Semántica

### 5.1 Concepto de Embeddings

Los embeddings son representaciones vectoriales densas de texto:

**Proceso:**
```
Texto → Modelo de Embedding → Vector [768 dimensiones]
"La IA cambia..." → [0.23, -0.45, 0.12, ...]
```

**Propiedades:**
- Textos similares tienen vectores cercanos
- Capturan significado semántico
- Dimensionalidad típica: 384-1536
- Permiten búsqueda por similitud

### 5.2 Modelos de Embedding Populares

| Modelo | Dimensiones | Idioma | Uso |
|--------|-------------|--------|-----|
| sentence-transformers/paraphrase-multilingual | 768 | Multi | General |
| text-embedding-ada-002 (OpenAI) | 1536 | Multi | Alta calidad |
| intfloat/multilingual-e5-large | 1024 | Multi | Balance |
| BAAI/bge-m3 | 1024 | Multi | Estado del arte |

### 5.3 Estrategias de Búsqueda

#### Similitud Coseno

Métrica más común para comparar embeddings:

```
similitud = (A · B) / (||A|| * ||B||)
Rango: [-1, 1] donde 1 es idéntico
```

#### Distancia Euclidiana

Alternativa basada en distancia geométrica:

```
distancia = √Σ(ai - bi)²
Menor distancia = mayor similitud
```

#### Producto Interno

Más rápido pero requiere vectores normalizados:

```
similitud = Σ(ai * bi)
```

### 5.4 Workflow RAG Completo

```
1. INDEXACIÓN (Offline)
   ├─ Cargar corpus_rag.json
   ├─ Generar embeddings para cada documento
   ├─ Almacenar en base vectorial (FAISS, Chroma, etc.)
   └─ Crear índice para búsqueda rápida

2. QUERY (Online)
   ├─ Recibir pregunta del usuario
   ├─ Generar embedding de la pregunta
   ├─ Buscar top-k documentos más similares
   └─ Recuperar documentos con metadata

3. GENERACIÓN
   ├─ Construir prompt con documentos recuperados
   ├─ Enviar a LLM (GPT, Claude, Llama, etc.)
   ├─ Generar respuesta fundamentada
   └─ Citar fuentes cuando sea apropiado
```

## 6. Bases de Datos Vectoriales

### 6.1 Opciones Populares

#### FAISS (Facebook AI)

**Características:**
- Librería de código abierto
- Extremadamente rápida
- Requiere implementación manual de persistencia
- Ideal para experimentación

**Ventajas:**
- Búsqueda en milisegundos
- Soporta millones de vectores
- Múltiples algoritmos de indexación
- Sin dependencias externas

#### ChromaDB

**Características:**
- Base de datos especializada en embeddings
- Persistencia automática
- API simple y pythónica
- Integración con LangChain

**Ventajas:**
- Fácil de usar
- Maneja metadata automáticamente
- Filtraje por metadata
- Escalabilidad moderada

#### Pinecone

**Características:**
- Servicio cloud especializado
- Escalabilidad automática
- API RESTful
- Pago por uso

**Ventajas:**
- Sin infraestructura
- Alta disponibilidad
- Actualizaciones en tiempo real
- Soporte empresarial

#### Weaviate

**Características:**
- Base de datos vectorial open source
- GraphQL API
- Búsqueda híbrida (vectorial + keyword)
- Self-hosted o cloud

**Ventajas:**
- Combina búsqueda semántica y tradicional
- Esquema flexible
- Multitenancy
- Modelos de embedding integrados

### 6.2 Comparación de Rendimiento

| Base de Datos | Velocidad | Escalabilidad | Complejidad | Costo |
|---------------|-----------|---------------|-------------|-------|
| FAISS | Muy alta | Alta | Media | Gratis |
| ChromaDB | Alta | Media | Baja | Gratis |
| Pinecone | Alta | Muy alta | Baja | Pago |
| Weaviate | Alta | Alta | Media | Gratis/Pago |

### 6.3 Integración con Formatos

**Para corpus_simple.json:**
```python
# Indexar solo el texto
vectorstore.add_texts(
    texts=[doc['texto'] for doc in corpus],
    metadatas=[doc['metadata'] for doc in corpus]
)
```

**Para corpus_rag.json:**
```python
# Elección entre simple o enriquecido
vectorstore.add_texts(
    texts=[doc['enriched_content'] for doc in corpus],
    metadatas=[doc['metadata'] for doc in corpus],
    ids=[doc['id'] for doc in corpus]
)
```

## 7. Optimizaciones y Estrategias

### 7.1 Enriquecimiento Contextual

**Ventajas:**
- Mejora precisión en búsquedas categorizadas
- Ayuda al modelo a entender contexto
- Permite filtrado implícito

**Desventajas:**
- Puede sesgar la búsqueda
- Incrementa longitud de documentos
- Puede afectar similitud entre categorías

**Cuándo Usar:**
- Dataset con categorías bien definidas
- Queries que mencionan temas específicos
- Necesidad de filtrado por metadata

### 7.2 Chunking (Segmentación)

Para documentos largos, considerar segmentación:

**Estrategias:**

```python
# Por longitud fija
chunks = [texto[i:i+500] for i in range(0, len(texto), 400)]
# Overlap de 100 caracteres

# Por oraciones
sentences = texto.split('. ')
chunks = ['. '.join(sentences[i:i+5]) for i in range(0, len(sentences), 4)]
# Grupos de 5 oraciones con overlap de 1

# Por párrafos semánticos
# Usa NLP para detectar cambios de tema
```

**Consideraciones:**
- Chunks más pequeños: mayor precisión, menos contexto
- Chunks más grandes: más contexto, menos precisión
- Overlap previene pérdida de información en fronteras

### 7.3 Filtrado por Metadata

Combinar búsqueda semántica con filtros estructurados:

```python
# Ejemplo con ChromaDB
results = collection.query(
    query_texts=["pregunta del usuario"],
    where={"tema": "Generación Z y crisis de sentido"},
    n_results=5
)
```

**Ventajas:**
- Restringe búsqueda a subconjunto relevante
- Mejora precisión en dominios específicos
- Reduce ruido en resultados

### 7.4 Reranking

Mejora los resultados iniciales:

**Proceso:**
1. Recuperar top-20 con búsqueda vectorial rápida
2. Aplicar modelo más sofisticado a los 20
3. Reordenar y seleccionar top-5 finales

**Modelos de Reranking:**
- Cross-encoders (más precisos pero lentos)
- Scoring por engagement
- Diversidad temática
- Recencia temporal

### 7.5 Híbrido: Vectorial + Keyword

Combina búsqueda semántica con búsqueda tradicional:

```
Score_final = α * Score_vectorial + (1-α) * Score_BM25
```

**Ventajas:**
- Captura matches exactos importantes
- Complementa debilidades de cada método
- Mejor para queries con términos técnicos

## 8. Casos de Uso Específicos

### 8.1 Asistente Conversacional Temático

**Escenario:**
- Chatbot que responde sobre temas generacionales
- Necesita fundamentar respuestas en opiniones reales
- Debe citar fuentes cuando sea relevante

**Configuración Recomendada:**
- Formato: `corpus_rag.json` con enriched_content
- Base vectorial: ChromaDB
- Top-k: 3-5 documentos
- Reranking por engagement

**Prompt Template:**
```
Contexto recuperado:
{documento_1}
{documento_2}
{documento_3}

Pregunta del usuario: {query}

Instrucciones: Responde basándote en el contexto proporcionado.
Si mencionas ideas del contexto, cita la fuente.
```

### 8.2 Análisis de Sentimientos por Tema

**Escenario:**
- Investigación sobre opiniones generacionales
- Necesita agrupar documentos por tema y sentimiento
- Busca patrones en el discurso

**Configuración Recomendada:**
- Formato: `corpus_simple.json`
- Filtrado por metadata (tema + sentimiento)
- Clustering adicional con embeddings

### 8.3 Sistema de Recomendación de Contenido

**Escenario:**
- Sugerir publicaciones similares
- Basado en el contenido que un usuario está leyendo
- Considera engagement para ordenar

**Configuración Recomendada:**
- Formato: `corpus_rag.json`
- Búsqueda por similitud del documento actual
- Reranking por engagement y diversidad temática
- Excluir mismo tema si se busca variedad

### 8.4 Búsqueda Semántica Directa

**Escenario:**
- Motor de búsqueda interno
- Usuario busca contenido específico
- No necesita generación, solo recuperación

**Configuración Recomendada:**
- Formato: `corpus_simple.json`
- Búsqueda híbrida (vectorial + BM25)
- Mostrar snippets con highlighting
- Ordenar por relevancia y fecha

## 9. Mejores Prácticas

### 9.1 Preparación de Datos

**Limpieza:**
- Eliminar duplicados exactos
- Corregir errores de encoding
- Normalizar espacios en blanco
- Verificar integridad de metadatos

**Validación:**
- Confirmar que todos los campos requeridos existen
- Verificar tipos de datos
- Detectar valores nulos o inconsistentes
- Validar rangos numéricos

### 9.2 Elección de Formato

**Usar corpus_simple.json cuando:**
- Modelo de embeddings maneja metadata separadamente
- Necesitas máxima pureza semántica
- Experimentas con diferentes enriquecimientos

**Usar corpus_enriquecido.json cuando:**
- Queries mencionan categorías explícitamente
- Dataset tiene categorías muy distintas
- Necesitas filtrado implícito

**Usar corpus_rag.json cuando:**
- Implementas sistema RAG completo
- Necesitas flexibilidad para elegir versión
- Requieres métricas adicionales (engagement)

### 9.3 Optimización de Búsqueda

**Top-k Apropiado:**
- Queries específicas: k=3-5
- Queries amplias: k=10-20
- Con reranking: recuperar 2-3x más

**Threshold de Similitud:**
- Establecer mínimo de similitud (ej. 0.7)
- Si no hay matches suficientes, indicar al usuario
- Evitar respuestas basadas en documentos irrelevantes

### 9.4 Actualización del Corpus

**Estrategias:**
- Incremental: Añadir nuevos documentos sin reindexar todo
- Batch: Reindexar periódicamente (semanal/mensual)
- Versioning: Mantener múltiples versiones del índice

**Consideraciones:**
- Reindexar puede tardar con corpus grandes
- Mantener consistencia entre datos y embeddings
- Planificar ventanas de mantenimiento

## 10. Evaluación de Sistemas RAG

### 10.1 Métricas de Recuperación

**Precisión (Precision):**
```
Precisión = Documentos Relevantes Recuperados / Total Documentos Recuperados
```
- Mide calidad de los resultados
- Alta precisión: pocos falsos positivos

**Recall (Cobertura):**
```
Recall = Documentos Relevantes Recuperados / Total Documentos Relevantes
```
- Mide completitud de los resultados
- Alto recall: no se pierden documentos importantes

**F1-Score:**
```
F1 = 2 * (Precisión * Recall) / (Precisión + Recall)
```
- Balance entre precisión y recall

**MRR (Mean Reciprocal Rank):**
```
MRR = (1/N) * Σ(1/rank_primer_relevante)
```
- Mide qué tan arriba aparece el primer resultado relevante

### 10.2 Métricas de Generación

**Faithfulness:**
- Respuesta fundamentada en documentos recuperados
- Sin alucinaciones o información externa

**Answer Relevance:**
- Respuesta directamente relacionada con la pregunta
- No divaga ni incluye información innecesaria

**Context Relevance:**
- Documentos recuperados son pertinentes
- No contienen ruido irrelevante

### 10.3 Evaluación Práctica

**Dataset de Prueba:**
- Crear conjunto de queries representativas
- Anotar documentos relevantes para cada query
- Evaluar respuestas generadas

**Ejemplo:**
```python
test_queries = [
    {
        "query": "¿Cómo afecta la IA a nuestra autonomía?",
        "relevant_docs": ["doc_5", "doc_13", "doc_49"],
        "expected_themes": ["IA y pérdida de autonomía humana"]
    },
    # más casos...
]
```

## 11. Arquitectura de Sistema RAG Completo

### 11.1 Componentes del Sistema

```
┌─────────────────────────────────────────────────────┐
│              SISTEMA RAG COMPLETO                   │
└─────────────────────────────────────────────────────┘

┌──────────────┐       ┌──────────────┐       ┌──────────────┐
│   Ingesta    │──────▶│  Indexación  │──────▶│ Almacenamiento│
└──────────────┘       └──────────────┘       └──────────────┘
      │                      │                       │
      │                      │                       │
 CSV Parser           Embedding Model          Vector DB
 Limpieza            (sentence-transformers)   (ChromaDB)
 Validación          Batch processing          Persistencia

                            ▼

┌──────────────┐       ┌──────────────┐       ┌──────────────┐
│    Query     │──────▶│  Retrieval   │──────▶│  Generation  │
└──────────────┘       └──────────────┘       └──────────────┘
      │                      │                       │
      │                      │                       │
 User Input           Similarity Search         LLM
 Embedding            Top-k Selection          Prompt Builder
 Preprocessing        Reranking                Response Format

                            ▼

                    ┌──────────────┐
                    │   Response   │
                    └──────────────┘
                          │
                    Answer + Sources
                    Confidence Score
                    Citations
```

### 11.2 Implementación con LangChain

```python
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

# 1. Cargar embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
)

# 2. Crear vector store
vectorstore = Chroma(
    persist_directory="./chroma_db",
    embedding_function=embeddings
)

# 3. Cargar documentos
import json
with open('corpus_rag.json') as f:
    docs = json.load(f)

# 4. Indexar
vectorstore.add_texts(
    texts=[d['enriched_content'] for d in docs],
    metadatas=[d['metadata'] for d in docs],
    ids=[d['id'] for d in docs]
)

# 5. Crear cadena RAG
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(temperature=0),
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 5})
)

# 6. Query
response = qa_chain.run("¿Cómo afecta la IA a la autonomía humana?")
```

### 11.3 Stack Tecnológico Recomendado

**Backend:**
- FastAPI o Flask para API
- Celery para procesamiento asíncrono
- Redis para caché

**Storage:**
- ChromaDB o Weaviate para vectores
- PostgreSQL para metadata estructurada
- S3 para respaldo de datos

**Embeddings:**
- sentence-transformers local
- OpenAI embeddings para producción
- Cohere embeddings como alternativa

**LLM:**
- OpenAI GPT-4 para calidad máxima
- Anthropic Claude para largo contexto
- Llama 2/3 local para privacidad